# Chapter 1: Your Deep Learning Journey

The following code comes from the fastai book found [here](https://github.com/fastai/fastbook). Sprinkled throughout the jupyter notebook are notes and ideas that come from the book and my interpretation of what is being said. Enjoy!

In [1]:
import fastbook
fastbook.setup_book()

/home/ubuntu/anaconda3/envs/fastai/lib/python3.7/site-packages/fastbook/__init__.py:18: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
from fastbook import *
import ipywidgets as widgets
from IPython.display import Image

## Cats and Dogs: Image Classification 

In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.PETS)/'images'
path

In [ ]:
def is_cat(x): return x[0].isupper()

In [ ]:
dls = ImageDataLoaders.from_name_func(path,
                                     get_image_files(path), valid_pct=0.2,
                                     seed = 42, label_func=is_cat, item_tfms=Resize(224))

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)

In [ ]:
learn.fine_tune(1)

In [ ]:
img = PILImage.create(image_cat())
img.to_thumb(192)

In [ ]:
uploader = widgets.FileUpload()
uploader

In [ ]:
img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

## Localizing Objects in a picture: Image Segmentation

In [ ]:
path = untar_data(URLs.CAMVID_TINY)

In [ ]:
dls = SegmentationDataLoaders.from_label_func(path,
                                             bs=8, fnames=get_image_files(path/"images"),
                                             label_func = lambda o: path/'labels'/f'{o.stem}_P{o.suffix}',
                                             codes=np.loadtxt(path/'codes.txt', dtype=str)
                                             )

In [ ]:
learn = unet_learner(dls, resnet34)

In [ ]:
learn.fine_tune(8)

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))

## Movie Reviews: Sentiment Classification

In [ ]:
from fastai.text.all import *

In [ ]:
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid="test")

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn.fine_tune(4, 1e-2)

In [ ]:
learn.predict("I really liked that movie!")

In [ ]:
learn.predict("I'm just making this review in hopes someone at Disney sees this. The digital dogs look bad. Your producers have what I call 'digital blindness' where they can't see which digital effects look much worse than practical effects. Please use practical effects as often as possible.")

In [ ]:
doc(learn.predict)

## Predict a high-income earner: Tabular Data

In [ ]:
from fastai.tabular.all import *
path = untar_data(URLs.ADULT_SAMPLE)

In [ ]:
dls = TabularDataLoaders.from_csv(path/"adult.csv", path=path, y_names="salary",
                                 cat_names = ["workclass", "education", "marital-status", "occupation", 
                                             "relationship", "race"],
                                 cont_names = ["age", "fnlwgt", "education-num"],
                                 procs = [Categorify, FillMissing, Normalize])

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(3)

## Predict Movies people might like: Recommendation System

In [3]:
from fastai.collab import *

In [4]:
path = untar_data(URLs.ML_SAMPLE)

In [5]:
dls = CollabDataLoaders.from_csv(path/'ratings.csv')
learn = collab_learner(dls, y_range=(0.5, 5.5))
learn.fine_tune(10)

epoch,train_loss,valid_loss,time
0,1.503958,1.418632,00:00


epoch,train_loss,valid_loss,time
0,1.376203,1.359422,00:00
1,1.252138,1.180762,00:00
2,1.017709,0.880817,00:00
3,0.798113,0.741172,00:00
4,0.688681,0.708689,00:00
5,0.648084,0.697439,00:00
6,0.631074,0.693731,00:00
7,0.608035,0.691561,00:00
8,0.609987,0.691219,00:00
9,0.607285,0.691045,00:00


In [6]:
learn.show_results()

,userId,movieId,rating,rating_pred
0,63.0,48.0,5.0,2.935433
1,70.0,36.0,4.0,4.040259
2,65.0,92.0,4.5,4.266208
3,47.0,98.0,5.0,4.345912
4,4.0,83.0,3.5,4.335626
5,4.0,38.0,4.5,4.221953
6,59.0,60.0,5.0,4.432968
7,86.0,82.0,4.0,3.797124
8,55.0,86.0,4.5,3.959364
